<!--TITLE:Custom Convnets-->
# Introduction #

Now that you've seen the layers a convnet uses to extract features, it's time to put them together and build a network of your own!

<figure>
<!-- <img src="./images/2-filter-detect-condense.png" width="400" alt="Extraction as a sequence of blocks."> -->
<img src="https://i.imgur.com/vBatPfn.png" width="400" alt="Extraction as a sequence of blocks.">
</figure>

# Convolutional Blocks #

Arrange them into blocks.

<figure>
<!-- <img src="./images/2-block-crp.png" width="400" alt="A kind of extraction block: convolution, ReLU, pooling."> -->
<img src="https://i.imgur.com/8D6IhEw.png" width="400" alt="A kind of extraction block: convolution, ReLU, pooling.">
</figure>

Modern convolutional networks comprise many of these blocks within their base:

<figure>
<!-- <img src="./images/2-block-seq.png" width="1200" alt="A sequence of extraction blocks."> -->
<img src="https://i.imgur.com/pr8VwCZ.png" width="1200" alt="A sequence of extraction blocks.">
</figure>

Each block represents a round of feature extraction, and it is by composing these blocks that the features the network learns become increasingly refined. This deep structure is what enables modern convnets to learn very complex visual features.

# Simple to Refined #

More extraction makes better features.

<figure>
<img src="https://i.imgur.com/VqmC1rm.png" alt="Features extracted from an image of a car, from simple to refined." width=1200>
</figure>

# Example - Design a Convnet #

Here is a diagram of the model we want to define:

<figure>
<!-- <img src="./images/2-convmodel-1.png" width="200" alt="Diagram of a convolutional model."> -->
<img src="https://i.imgur.com/dMO4nRb.png" width="200" alt="Diagram of a convolutional model.">
</figure>

## Step 1 - Load Data ##

The next hidden cell will load our dataset.

In [ ]:
#$HIDE_INPUT$
# Imports
import os
import warnings
import numpy as np
import visiontools
from visiontools import StanfordCars
import tensorflow as tf
import tensorflow_datasets as tfds

# Reproducibility
def seed_everything(seed=31415):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'

seed = 31415
seed_everything(seed)
warnings.filterwarnings("ignore")

# Load training and validation sets
DATA_DIR = '/kaggle/input/stanford-cars-for-learn/'
(ds_train_, ds_valid_), ds_info = tfds.load(
    'stanford_cars/simple',
    split=['train', 'test'],
    shuffle_files=True,
    with_info=True,
    data_dir=DATA_DIR,
)
print(("Loaded {} training examples " +
       "and {} validation examples " +
       "with classes {}.").format(
           ds_info.splits['train'].num_examples,
           ds_info.splits['test'].num_examples,
           ds_info.features['label'].names))

# Create data pipeline
BATCH_SIZE = 16
AUTO = tf.data.experimental.AUTOTUNE
SIZE = [192, 192]
preprocess = visiontools.make_preprocessor(size=SIZE)

ds_train = (ds_train_
            .map(preprocess)
            .cache()
            .shuffle(ds_info.splits['train'].num_examples)
            .batch(BATCH_SIZE)
            .prefetch(AUTO))

ds_valid = (ds_valid_
            .map(preprocess)
            .cache()
            .shuffle(ds_info.splits['test'].num_examples)
            .batch(BATCH_SIZE)
            .prefetch(AUTO))

## Step 2 ##

We'll build it as a Keras `Sequential` model and stack a couple of dense layers on top for a classifier just like we did with the model in Lesson 1.

In [ ]:
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

model = keras.Sequential([

    # First Convolutional Block
    layers.Conv2D(filters=64,
                  kernel_size=5,
                  activation="relu",
                  padding='same'),
    layers.MaxPool2D(),

    # Second Convolutional Block    
    layers.Conv2D(filters=128,
                  kernel_size=3,
                  activation="relu",
                  padding='same'),
    layers.MaxPool2D(),

    # Third Convolutional Block    
    layers.Conv2D(filters=256,
                  kernel_size=3,
                  activation="relu",
                  padding='same'),
    layers.MaxPool2D(),

    # Classifier Head #
    layers.Flatten(),
    layers.Dense(units=8,
                 activation="relu"),
    layers.Dense(units=1,
                 activation="sigmoid"),
])

## Step 3 - Train ##

Training is just the same as before, too!

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['binary_accuracy']
)

history = model.fit(ds_train,
                    validation_data=ds_valid,
                    epochs=10)

In [ ]:
import pandas as pd

history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot();

# Conclusion #